# Gaze Pattern Analysis and LSTM Autoencoder Modeling

This notebook analyzes gaze patterns from the OneStop dataset and builds an LSTM autoencoder to learn representations of reading behavior.

## 1. Setup and Data Loading

In [1]:
!pip install pymovements
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.1/325.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.0
    Uninstalling wrapt-2.0.0:
      Successfully uninstalled wrapt-2.0.0
  Attempting uninstall: scipy
    Found existing installation:

In [2]:
import pymovements as pm
dataset = pm.Dataset('OneStop', path='data/OneStop')

dataset.download()

fixations_Paragraph.csv.zip: 0.00B [00:00, ?B/s]

Checking integrity of fixations_Paragraph.csv.zip


ia_Paragraph.csv.zip: 0.00B [00:00, ?B/s]

Checking integrity of ia_Paragraph.csv.zip
Extracting fixations_Paragraph.csv.zip to data/OneStop/precomputed_events


100%|██████████| 1/1 [00:50<00:00, 50.84s/it]


Extracting ia_Paragraph.csv.zip to data/OneStop/precomputed_reading_measures


100%|██████████| 1/1 [00:54<00:00, 54.06s/it]


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz
from math import atan2, degrees, isnan, sqrt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


## 2. Data Download and Initial Loading

In [4]:
# --- Configuration ---
# Set a more professional and publication-ready style for plots
sns.set_theme(style="whitegrid", context="talk")
plt.rcParams['figure.figsize'] = (12, 7)

# --- Load and Prepare the Data ---
print("Loading data... this might take a moment.")
try:
    # Using low_memory=False can help with mixed data types in large files.
    df = pd.read_csv('/content/data/OneStop/precomputed_events/fixations_Paragraph.csv', low_memory=False)
    print("Data loaded successfully.")
    print(f"Shape: {df.shape}")
    print(f"\nFirst few column names: {list(df.columns[:10])}")
except Exception as e:
    print(f"Error loading data: {e}")

# Quick verification of key columns
key_columns = ['CURRENT_FIX_DURATION', 'NEXT_SAC_AMPLITUDE', 'participant_id',
               'TRIAL_INDEX', 'word_length', 'wordfreq_frequency', 'difficulty_level',
               'CURRENT_FIX_INTEREST_AREA_ID', 'CURRENT_FIX_INTEREST_AREA_LABEL',
               'CURRENT_FIX_X', 'CURRENT_FIX_Y', 'CURRENT_FIX_INDEX']

print("\n--- Checking key columns ---")
for col in key_columns:
    if col in df.columns:
        print(f"✓ {col}")
    else:
        print(f"✗ {col} - MISSING!")

Loading data... this might take a moment.
Data loaded successfully.
Shape: (2400788, 290)

First few column names: ['participant_id', 'TRIAL_INDEX', 'CURRENT_FIX_ADJUSTED', 'CURRENT_FIX_BLINK_AROUND', 'CURRENT_FIX_BUTTON_0_PRESS', 'CURRENT_FIX_BUTTON_1_PRESS', 'CURRENT_FIX_BUTTON_2_PRESS', 'CURRENT_FIX_BUTTON_3_PRESS', 'CURRENT_FIX_BUTTON_4_PRESS', 'CURRENT_FIX_BUTTON_5_PRESS']

--- Checking key columns ---
✓ CURRENT_FIX_DURATION
✓ NEXT_SAC_AMPLITUDE
✓ participant_id
✓ TRIAL_INDEX
✓ word_length
✓ wordfreq_frequency
✓ difficulty_level
✓ CURRENT_FIX_INTEREST_AREA_ID
✓ CURRENT_FIX_INTEREST_AREA_LABEL
✓ CURRENT_FIX_X
✓ CURRENT_FIX_Y
✓ CURRENT_FIX_INDEX


## 3. Initial Data Analysis (Descriptive & Inferential)

In [5]:
def plot_scanpath(df, participant_id, trial_index):
    """
    Visualizes the scanpath for a single trial of a single participant.
    """
    trial_data = df[(df['participant_id'] == participant_id) & (df['TRIAL_INDEX'] == trial_index)]

    if trial_data.empty:
        print(f"No data found for participant {participant_id}, trial {trial_index}")
        return

    # Sort fixations in chronological order
    trial_data = trial_data.sort_values(by='CURRENT_FIX_INDEX')

    x = trial_data['CURRENT_FIX_X']
    y = trial_data['CURRENT_FIX_Y']
    duration = trial_data['CURRENT_FIX_DURATION']

    plt.figure(figsize=(16, 8))

    # Plot the saccades (lines connecting fixations)
    plt.plot(x, y, marker='', linestyle='-', color='gray', alpha=0.6)

    # Plot the fixations (circles sized by duration)
    scatter = plt.scatter(x, y, s=duration, c=np.arange(len(duration)), cmap='viridis', alpha=0.7, edgecolors='black')

    # Add text labels for the first few fixations to show the reading order
    for i in range(min(15, len(trial_data))):
        plt.text(x.iloc[i], y.iloc[i], str(i+1), ha='center', va='center', color='white', fontsize=10, fontweight='bold')

    plt.title(f'Scanpath for Participant: {participant_id}, Trial: {trial_index}')
    plt.xlabel('X Coordinate (pixels)')
    plt.ylabel('Y Coordinate (pixels)')

    # Invert y-axis as screen coordinates typically start from top-left
    plt.gca().invert_yaxis()
    plt.axis('equal')

    # Add a colorbar to show the time progression
    cbar = plt.colorbar(scatter, label='Fixation Sequence Order')

    plt.savefig(f"scanpath_{participant_id}_trial_{trial_index}.png")
    print(f"Saved plot: scanpath_{participant_id}_trial_{trial_index}.png")
    plt.close()


def perform_analysis(df):
    """
    Run all analyses on the loaded dataframe.
    """
    # --- 2. Descriptive Analysis: Understanding the Basics ---
    print("\n--- Running Descriptive Analysis ---")

    # Plotting the distribution of fixation durations
    plt.figure()
    sns.histplot(df['CURRENT_FIX_DURATION'], kde=True, bins=100, stat="density")
    plt.title('Distribution of Fixation Durations')
    plt.xlabel('Duration (ms)')
    plt.ylabel('Density')
    plt.xlim(0, 1000)
    plt.axvline(df['CURRENT_FIX_DURATION'].median(), color='red', linestyle='--', label=f"Median: {df['CURRENT_FIX_DURATION'].median():.0f} ms")
    plt.legend()
    plt.savefig("fixation_duration_distribution.png")
    print("Saved plot: fixation_duration_distribution.png")
    plt.close()

    # Plotting the distribution of saccade amplitudes
    plt.figure()
    # Convert 'NEXT_SAC_AMPLITUDE' to numeric, coercing errors, before filtering
    df['NEXT_SAC_AMPLITUDE'] = pd.to_numeric(df['NEXT_SAC_AMPLITUDE'], errors='coerce')
    valid_saccades = df['NEXT_SAC_AMPLITUDE'].dropna().loc[df['NEXT_SAC_AMPLITUDE'] > 0]
    sns.histplot(valid_saccades, kde=True, bins=100)
    plt.title('Distribution of Saccade Amplitudes')
    plt.xlabel('Amplitude (degrees of visual angle)')
    plt.ylabel('Count')
    plt.xlim(0, 20)
    plt.axvline(valid_saccades.median(), color='red', linestyle='--', label=f"Median: {valid_saccades.median():.2f} degrees")
    plt.legend()
    plt.savefig("saccade_amplitude_distribution.png")
    print("Saved plot: saccade_amplitude_distribution.png")
    plt.close()

    # --- 3. Scanpath Visualization ---
    print("\n--- Generating a Scanpath Visualization ---")
    example_participant = df['participant_id'].iloc[0]
    example_trial = df['TRIAL_INDEX'].iloc[0]
    plot_scanpath(df, example_participant, example_trial)

    # --- 4. Inferential Analysis: Testing Hypotheses ---
    print("\n--- Running Inferential Analysis ---")

    # Aggregate fixation data to word level
    word_level_df = df.groupby(['participant_id', 'TRIAL_INDEX', 'CURRENT_FIX_INTEREST_AREA_ID', 'CURRENT_FIX_INTEREST_AREA_LABEL']).agg(
        total_fixation_duration=('CURRENT_FIX_DURATION', 'sum'),
        fixation_count=('CURRENT_FIX_DURATION', 'count'),
        word_length=('word_length', 'first'),
        wordfreq_frequency=('wordfreq_frequency', 'first'),
        difficulty_level=('difficulty_level', 'first')
    ).reset_index()

    # Filter out very long words
    word_level_df = word_level_df[word_level_df['word_length'].between(1, 15)]

    # --- Plot 4a: Word Length Effect ---
    plt.figure()
    sns.regplot(data=word_level_df.sample(n=min(5000, len(word_level_df)), random_state=42),
                x='word_length',
                y='total_fixation_duration',
                scatter_kws={'alpha':0.2},
                line_kws={'color':'red'})
    plt.title('Word Length Effect: Longer Words Receive Longer Gaze Times')
    plt.xlabel('Word Length (characters)')
    plt.ylabel('Total Fixation Duration on Word (ms)')
    plt.ylim(0, 2000)
    plt.savefig("word_length_effect.png")
    print("Saved plot: word_length_effect.png")
    plt.close()

    # --- Word Frequency Effect ---
    print("\n--- Analyzing Word Frequency Effect ---")
    word_level_df['log_word_frequency'] = np.log1p(word_level_df['wordfreq_frequency'])

    plt.figure()
    sns.regplot(data=word_level_df.sample(n=min(5000, len(word_level_df)), random_state=42),
                x='log_word_frequency',
                y='total_fixation_duration',
                scatter_kws={'alpha':0.2},
                line_kws={'color':'green'})
    plt.title('Word Frequency Effect: Rarer Words Receive Longer Gaze Times')
    plt.xlabel('Log Word Frequency (higher is more frequent)')
    plt.ylabel('Total Fixation Duration on Word (ms)')
    plt.ylim(0, 2000)
    plt.savefig("word_frequency_effect.png")
    print("Saved plot: word_frequency_effect.png")
    plt.close()

    # --- Effect of Text Difficulty ---
    ###### Didnt find Elementry category ###### Need Fixation!
    print("\n--- Analyzing Effect of Text Difficulty ---")
    plt.figure()
    sns.boxplot(data=word_level_df,
                x='difficulty_level',
                y='total_fixation_duration',
                order=['Elementary', 'Adv'])
    plt.title('Effect of Text Difficulty on Gaze Time')
    plt.xlabel('Text Difficulty Level')
    plt.ylabel('Total Fixation Duration on Word (ms)')
    plt.ylim(0, 1000)
    plt.savefig("difficulty_effect.png")
    print("Saved plot: difficulty_effect.png")
    plt.close()

    print("\n--- Analysis Complete! ---")

In [6]:
perform_analysis(df)


--- Running Descriptive Analysis ---
Saved plot: fixation_duration_distribution.png
Saved plot: saccade_amplitude_distribution.png

--- Generating a Scanpath Visualization ---
Saved plot: scanpath_l42_2070_trial_1.png

--- Running Inferential Analysis ---
Saved plot: word_length_effect.png

--- Analyzing Word Frequency Effect ---
Saved plot: word_frequency_effect.png

--- Analyzing Effect of Text Difficulty ---
Saved plot: difficulty_effect.png

--- Analysis Complete! ---


In [7]:
def final_analysis(df):
    """
    Pre-Mortem analysis before preprocessing:
    - Compare reading regimes
    - Check data quality (blinks)
    """
    print("\n--- Running Final Analysis Before Preprocessing ---")

    # Create the aggregated word-level dataframe
    word_level_df = df.groupby(['participant_id', 'TRIAL_INDEX', 'CURRENT_FIX_INTEREST_AREA_ID', 'CURRENT_FIX_INTEREST_AREA_LABEL']).agg(
        total_fixation_duration=('CURRENT_FIX_DURATION', 'sum'),
        fixation_count=('CURRENT_FIX_DURATION', 'count'),
        word_length=('word_length', 'first'),
        wordfreq_frequency=('wordfreq_frequency', 'first'),
        difficulty_level=('difficulty_level', 'first'),
        # Add question_preview to know the reading condition
        question_preview=('question_preview', 'first')
    ).reset_index()

    word_level_df = word_level_df[word_level_df['word_length'].between(1, 15)]

    # --- Analysis 5a: Compare Reading Regimes ---
    print("\n--- Analyzing Reading Regimes ---")

    # Create a more readable 'regime' column from the boolean 'question_preview'
    word_level_df['reading_regime'] = word_level_df['question_preview'].apply(
        lambda x: 'Information-Seeking' if x else 'Ordinary Reading'
    )

    plt.figure()
    # A violin plot is great here as it shows the distribution shape
    sns.violinplot(data=word_level_df,
                   x='reading_regime',
                   y='total_fixation_duration')
    plt.title('Reading Behavior Differs Significantly by Task')
    plt.xlabel('Reading Regime')
    plt.ylabel('Total Fixation Duration on Word (ms)')
    plt.ylim(0, 1500)
    plt.savefig("regime_effect.png")
    print("Saved plot: regime_effect.png")
    plt.close()

    # Print summary statistics
    print("\nSummary Statistics by Reading Regime:")
    print(word_level_df.groupby('reading_regime')['total_fixation_duration'].describe())

    # --- Analysis 5b: Check Data Quality (Blinks) ---
    print("\n--- Data Quality Check: Blinks ---")

    # Blinks are not reading data and should be removed before modeling.
    # The 'CURRENT_FIX_BLINK_AROUND' column identifies fixations contaminated by blinks.
    blink_counts = df['CURRENT_FIX_BLINK_AROUND'].value_counts()
    total_fixations = len(df)

    # Calculate percentage of fixations with blinks before, after, or during
    blinks_around = blink_counts.get('BEFORE', 0) + blink_counts.get('AFTER', 0) + blink_counts.get('BOTH', 0)
    blink_percentage = (blinks_around / total_fixations) * 100

    print(f"\nTotal fixations in dataset: {total_fixations:,}")
    print(f"Fixations contaminated by blinks: {blinks_around:,} ({blink_percentage:.2f}%)")
    print("\nBlink breakdown:")
    print(blink_counts)
    print("\nRecommendation: These fixations should be filtered out during preprocessing.")

    # Visualize blink contamination
    plt.figure(figsize=(10, 6))
    blink_data = pd.DataFrame({
        'Category': ['Clean Fixations', 'Blink-Contaminated'],
        'Count': [total_fixations - blinks_around, blinks_around],
        'Percentage': [100 - blink_percentage, blink_percentage]
    })

    colors = ['#2ecc71', '#e74c3c']
    plt.bar(blink_data['Category'], blink_data['Count'], color=colors, alpha=0.7, edgecolor='black')
    plt.title('Data Quality: Blink Contamination in Fixations')
    plt.ylabel('Number of Fixations')

    # Add percentage labels on bars
    for i, row in blink_data.iterrows():
        plt.text(i, row['Count'], f"{row['Percentage']:.1f}%",
                ha='center', va='bottom', fontweight='bold', fontsize=12)

    plt.savefig("blink_contamination.png")
    print("\nSaved plot: blink_contamination.png")
    plt.close()

    print("\n--- Final Analysis Complete! ---")

In [8]:
final_analysis(df)


--- Running Final Analysis Before Preprocessing ---

--- Analyzing Reading Regimes ---
Saved plot: regime_effect.png

Summary Statistics by Reading Regime:
                        count        mean         std  min    25%    50%  \
reading_regime                                                             
Information-Seeking  705217.0  285.429159  230.840078  0.0  157.0  214.0   
Ordinary Reading     818456.0  309.204345  260.725657  0.0  163.0  230.0   

                       75%      max  
reading_regime                       
Information-Seeking  342.0  30260.0  
Ordinary Reading     372.0  29308.0  

--- Data Quality Check: Blinks ---

Total fixations in dataset: 2,400,788
Fixations contaminated by blinks: 252,136 (10.50%)

Blink breakdown:
CURRENT_FIX_BLINK_AROUND
NONE      2148652
AFTER      122796
BEFORE     122637
BOTH         6703
Name: count, dtype: int64

Recommendation: These fixations should be filtered out during preprocessing.

Saved plot: blink_contamination.png

---

## 4. Data Preprocessing

In [9]:
# Block 1: Modify Task Filter to Include Both Reading Regimes
# Purpose: Include both 'Ordinary Reading' and 'Information-Seeking' trials for broader applicability.

print("--- Block 1: Modifying Task Filter ---")

# Check if the 'question_preview' column exists before filtering
if 'question_preview' in df.columns:
    initial_rows = len(df)

    # Include both 'Ordinary Reading' (question_preview == False) and 'Information-Seeking' (question_preview == True)
    # By keeping all rows, we effectively include both.
    df_filtered_tasks = df.copy()

    print(f"Rows before filtering: {initial_rows}")
    print(f"Rows after including all reading regimes: {len(df_filtered_tasks)}")
    # No rows are removed in this step as we are including all tasks
else:
    print("Warning: 'question_preview' column not found. Proceeding with the full dataset.")
    df_filtered_tasks = df.copy() # Proceed with the full dataset if the column is missing

# --- End of Block 1 ---
print("\n" + "="*50 + "\n")

--- Block 1: Modifying Task Filter ---
Rows before filtering: 2400788
Rows after including all reading regimes: 2400788




In [10]:
# Block 2: Clean Data and Remove Artifacts (Blinks and Relaxed Duration Filter)
# Purpose: Remove blinks and apply a relaxed filter for physiologically plausible fixations.

print("--- Block 2: Cleaning Data and Removing Artifacts ---")

initial_rows = len(df_filtered_tasks)

# 1. Remove Blinks: Keep only rows where no blink was detected around the fixation.
df_cleaned = df_filtered_tasks[df_filtered_tasks['CURRENT_FIX_BLINK_AROUND'] == 'NONE'].copy()
print(f"Removed {initial_rows - len(df_cleaned)} rows with blinks.")

# 2. Relaxed Filter by Duration: Remove fixations that are too short or too long.
rows_before_duration_filter = len(df_cleaned)
df_cleaned = df_cleaned[df_cleaned['CURRENT_FIX_DURATION'].between(50, 1500)].copy()
print(f"Removed {rows_before_duration_filter - len(df_cleaned)} rows based on relaxed duration filter (50ms-1500ms).")

print(f"\nTotal rows remaining after cleaning: {len(df_cleaned)}")

# --- End of Block 2 ---
print("\n" + "="*50 + "\n")

--- Block 2: Cleaning Data and Removing Artifacts ---
Removed 252136 rows with blinks.
Removed 26688 rows based on relaxed duration filter (50ms-1500ms).

Total rows remaining after cleaning: 2121964




In [12]:
# Block 3: Feature Engineering and Selection (New Feature Set)
# Purpose: Select the core features for modeling reading dynamics, including the new IS_REGRESSION feature.

print("--- Block 3: Feature Engineering and Selection ---")

# Define the new set of features for our model
feature_columns = [
    'participant_id',           # For grouping
    'TRIAL_INDEX',              # For grouping
    'CURRENT_FIX_INDEX',        # Needed for sorting to calculate IS_REGRESSION
    'CURRENT_FIX_DURATION',
    'CURRENT_FIX_X',
    'CURRENT_FIX_Y',
    'PREVIOUS_SAC_AMPLITUDE'
    # PREVIOUS_SAC_AVG_VELOCITY is removed
]

# Create the new DataFrame with only the selected features
df_features = df_cleaned[feature_columns].copy()

# Convert problematic columns to numeric, coercing errors to NaN
df_features['PREVIOUS_SAC_AMPLITUDE'] = pd.to_numeric(df_features['PREVIOUS_SAC_AMPLITUDE'], errors='coerce')

# Handle missing values for saccade amplitude (occurs at the start of each trial)
# We fill with 0 as there was no preceding saccade.
df_features['PREVIOUS_SAC_AMPLITUDE'].fillna(0, inplace=True)

# Implement the new IS_REGRESSION feature
print("Calculating IS_REGRESSION feature...")

# Sort data by participant, trial, and fixation index to ensure correct sequence
df_features = df_features.sort_values(by=['participant_id', 'TRIAL_INDEX', 'CURRENT_FIX_INDEX']) # Ensure CURRENT_FIX_INDEX is available or add it

# Calculate previous X coordinate within each trial
df_features['PREVIOUS_FIX_X'] = df_features.groupby(['participant_id', 'TRIAL_INDEX'])['CURRENT_FIX_X'].shift(1)

# Calculate IS_REGRESSION
# IS_REGRESSION = 1 if CURRENT_FIX_X < PREVIOUS_FIX_X, 0 otherwise
# For the first fixation of each trial, PREVIOUS_FIX_X will be NaN, so IS_REGRESSION will be 0
df_features['IS_REGRESSION'] = ((df_features['CURRENT_FIX_X'] < df_features['PREVIOUS_FIX_X']) & ~df_features['PREVIOUS_FIX_X'].isna()).astype(int)

# Drop the temporary 'PREVIOUS_FIX_X' column
df_features.drop(columns=['PREVIOUS_FIX_X'], inplace=True)

# Add IS_REGRESSION to the feature columns list for the model
model_feature_names = [
    'CURRENT_FIX_DURATION', 'CURRENT_FIX_X', 'CURRENT_FIX_Y',
    'PREVIOUS_SAC_AMPLITUDE', 'IS_REGRESSION'
]

print("New feature 'IS_REGRESSION' calculated.")
print("Selected features for the model:")
print(df_features[model_feature_names].columns.tolist())
print("\nFirst 5 rows of the feature-engineered DataFrame (including IS_REGRESSION):")
print(df_features[model_feature_names].head())
print(f"\nAny remaining NaNs? {df_features[model_feature_names].isnull().sum().sum() == 0}")


# --- End of Block 3 ---
print("\n" + "="*50 + "\n")

--- Block 3: Feature Engineering and Selection ---


/tmp/ipython-input-3402703691.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_features['PREVIOUS_SAC_AMPLITUDE'].fillna(0, inplace=True)


Calculating IS_REGRESSION feature...
New feature 'IS_REGRESSION' calculated.
Selected features for the model:
['CURRENT_FIX_DURATION', 'CURRENT_FIX_X', 'CURRENT_FIX_Y', 'PREVIOUS_SAC_AMPLITUDE', 'IS_REGRESSION']

First 5 rows of the feature-engineered DataFrame (including IS_REGRESSION):
         CURRENT_FIX_DURATION  CURRENT_FIX_X  CURRENT_FIX_Y  \
2212059                   221          384.5          207.4   
2212060                   287          444.2          216.8   
2212061                   156          565.4          203.9   
2212064                   225          590.2          217.9   
2212065                   142          825.1          221.5   

         PREVIOUS_SAC_AMPLITUDE  IS_REGRESSION  
2212059                    0.00              0  
2212060                    1.01              0  
2212061                    2.29              0  
2212064                    1.16              0  
2212065                    4.52              0  

Any remaining NaNs? True




In [14]:
# Block 4: Create Gaze Sequences
# Purpose: Convert the continuous list of fixations into fixed-length, overlapping sequences.
# This creates the samples (X) for our sequence model.

print("--- Block 4: Creating Gaze Sequences ---")

# Define sequence parameters
SEQUENCE_LENGTH = 20
STEP = 5 # This creates overlapping sequences for more training data

# Group data by each unique trial for each participant
grouped = df_features.groupby(['participant_id', 'TRIAL_INDEX'])

all_sequences = []

# The features we want in our final sequence arrays (excluding identifiers)
model_feature_names = [
    'CURRENT_FIX_DURATION', 'CURRENT_FIX_X', 'CURRENT_FIX_Y',
    'PREVIOUS_SAC_AMPLITUDE', 'IS_REGRESSION' # Updated feature list
]

for name, group in grouped:
    # Get the feature values for the current trial
    trial_features = group[model_feature_names].values

    # Use a sliding window to create sequences
    if len(trial_features) >= SEQUENCE_LENGTH:
        for i in range(0, len(trial_features) - SEQUENCE_LENGTH + 1, STEP):
            sequence = trial_features[i:i + SEQUENCE_LENGTH]
            all_sequences.append(sequence)

# Convert the list of sequences to a single NumPy array
X = np.array(all_sequences)

print(f"Total number of sequences created: {X.shape[0]}")
print(f"Shape of each sequence (Sequence Length, Num Features): {X.shape[1:]}")

# --- End of Block 4 ---
print("\n" + "="*50 + "\n")

--- Block 4: Creating Gaze Sequences ---
Total number of sequences created: 342893
Shape of each sequence (Sequence Length, Num Features): (20, 5)




In [15]:
# Block 5: Data Splitting and Normalization
# Purpose: Split the data by participant and normalize features for optimal model training.

print("--- Block 5: Data Splitting and Normalization ---")

# 1. Get unique participants to split on
unique_participants = df_features['participant_id'].unique()

# 2. Split participants into train, validation, and test sets
train_participants, test_participants = train_test_split(unique_participants, test_size=0.2, random_state=42)
val_participants, test_participants = train_test_split(test_participants, test_size=0.5, random_state=42)

print(f"Training participants: {len(train_participants)}")
print(f"Validation participants: {len(val_participants)}")
print(f"Test participants: {len(test_participants)}")

# 3. Create a mapping from sequence index to participant_id
# This is a bit more complex but ensures a correct split.
participant_map = []
for name, group in grouped:
    trial_len = len(group)
    if trial_len >= SEQUENCE_LENGTH:
        num_sequences_in_trial = (trial_len - SEQUENCE_LENGTH) // STEP + 1
        participant_map.extend([name[0]] * num_sequences_in_trial)

# 4. Create datasets based on participant splits
train_indices = [i for i, pid in enumerate(participant_map) if pid in train_participants]
val_indices = [i for i, pid in enumerate(participant_map) if pid in val_participants]
test_indices = [i for i, pid in enumerate(participant_map) if pid in test_participants]

X_train = X[train_indices]
X_val = X[val_indices]
X_test = X[test_indices]

print(f"\nShape of X_train: {X_train.shape}")
print(f"Shape of X_val: {X_val.shape}")
print(f"Shape of X_test: {X_test.shape}")

# 5. Normalize the data
# Reshape data to 2D for the scaler
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

# Initialize and fit scaler ONLY on training data
scaler = StandardScaler()
scaler.fit(X_train_reshaped)

# Transform all datasets
X_train_scaled = scaler.transform(X_train_reshaped)
X_val_scaled = scaler.transform(X_val_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

# Reshape data back to 3D sequences
X_train = X_train_scaled.reshape(X_train.shape)
X_val = X_val_scaled.reshape(X_val.shape)
X_test = X_test_scaled.reshape(X_test.shape)

print("\nData has been successfully normalized.")
print(f"Mean of X_train (should be ~0): {X_train.mean():.2f}")
print(f"Std Dev of X_train (should be ~1): {X_train.std():.2f}")

# Optional: Save the processed data and the scaler for later use
np.save('X_train.npy', X_train)
np.save('X_val.npy', X_val)
np.save('X_test.npy', X_test)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("\nProcessed data and scaler have been saved to files.")

# --- End of Block 5 ---
print("\n" + "="*50 + "\n")
print("Preprocessing complete!")

--- Block 5: Data Splitting and Normalization ---
Training participants: 288
Validation participants: 36
Test participants: 36

Shape of X_train: (275632, 20, 5)
Shape of X_val: (35496, 20, 5)
Shape of X_test: (31765, 20, 5)

Data has been successfully normalized.
Mean of X_train (should be ~0): -0.00
Std Dev of X_train (should be ~1): 1.00

Processed data and scaler have been saved to files.


Preprocessing complete!


## 5. Model Definition and Training

In [16]:
# Block 1: Create TensorFlow Data Pipeline with Masking
# Purpose: Create an efficient data pipeline that handles batching, shuffling, and
# applying the masking "on-the-fly" to our sequences.

print("--- Block 1: Creating Data Pipeline ---")

# Define masking parameters
MASK_FRACTION = 0.15 # Percentage of timesteps to mask in each sequence

def mask_sequence(sequence):
    """
    Takes a single sequence and returns the masked version and the original.
    The original sequence will serve as the target (y_true).
    """
    masked_sequence = sequence.copy()
    seq_len, n_features = sequence.shape

    # Calculate how many timesteps to mask
    n_to_mask = int(np.ceil(seq_len * MASK_FRACTION))

    # Randomly choose indices to mask
    mask_indices = np.random.choice(seq_len, n_to_mask, replace=False)

    # Apply mask (set feature values to 0)
    masked_sequence[mask_indices] = 0.0

    return masked_sequence, sequence

def tf_masking_generator(dataset):
    """A generator function that yields masked and original sequences."""
    for sequence in dataset:
        yield mask_sequence(sequence)

# Create TensorFlow datasets
# Note: The output signature specifies the data types and shapes for our generator
output_signature = (
    tf.TensorSpec(shape=(X_train.shape[1], X_train.shape[2]), dtype=tf.float32),
    tf.TensorSpec(shape=(X_train.shape[1], X_train.shape[2]), dtype=tf.float32)
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: tf_masking_generator(X_train),
    output_signature=output_signature
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: tf_masking_generator(X_val),
    output_signature=output_signature
)

# Configure the datasets for performance
BATCH_SIZE = 256
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Data pipelines created.")
print("Each batch will provide (masked_sequences, original_sequences)")

# --- End of Block 1 ---
print("\n" + "="*50 + "\n")

--- Block 1: Creating Data Pipeline ---
Data pipelines created.
Each batch will provide (masked_sequences, original_sequences)




In [17]:
# Block 2: Define the LSTM Autoencoder Model
# Purpose: Build the sequence-to-sequence autoencoder based on our plan.

print("--- Block 2: Defining the LSTM Autoencoder Model ---")

# Hyperparameters
SEQUENCE_LENGTH = X_train.shape[1]
N_FEATURES = X_train.shape[2]
LATENT_DIM = 64 # Size of the compressed context vector

# --- Encoder ---
encoder_inputs = Input(shape=(SEQUENCE_LENGTH, N_FEATURES), name='encoder_input')
# Using two stacked LSTMs for the encoder
encoder_lstm1 = LSTM(LATENT_DIM * 2, return_sequences=True, name='encoder_lstm_1')(encoder_inputs)
# The final encoder LSTM returns only the last hidden state (the context vector)
encoder_lstm2 = LSTM(LATENT_DIM, return_sequences=False, name='encoder_lstm_2')(encoder_lstm1)
encoder_outputs = encoder_lstm2

# This defines the encoder model which we will save later
encoder = Model(encoder_inputs, encoder_outputs, name='gaze_encoder')

# --- Decoder ---
# The decoder's input is the context vector repeated for each timestep
decoder_inputs = RepeatVector(SEQUENCE_LENGTH, name='repeat_vector')(encoder_outputs)
# Two stacked LSTMs for the decoder, mirroring the encoder
decoder_lstm1 = LSTM(LATENT_DIM * 2, return_sequences=True, name='decoder_lstm_1')(decoder_inputs)
decoder_lstm2 = LSTM(N_FEATURES, return_sequences=True, name='decoder_lstm_2')(decoder_lstm1)
# The output layer maps back to our 5 original features
decoder_outputs = TimeDistributed(Dense(N_FEATURES, activation='linear'), name='time_distributed_output')(decoder_lstm2)

# --- Autoencoder ---
# The full model maps the masked input to the reconstructed output
autoencoder = Model(encoder_inputs, decoder_outputs, name='lstm_autoencoder')

# We'll define a custom loss in the next block
autoencoder.compile(optimizer='adam')

print("Model architecture defined.")
autoencoder.summary()

# --- End of Block 2 ---
print("\n" + "="*50 + "\n")

--- Block 2: Defining the LSTM Autoencoder Model ---
Model architecture defined.


Model: "lstm_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 20, 5)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_lstm_1 (LSTM)           │ (None, 20, 128)        │        68,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_lstm_2 (LSTM)           │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 20, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_lstm_1 (LSTM)           │ (None, 20, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_lstm_2 (LSTM)           │ (None, 20, 5)          │         2,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_output         │ (None, 20, 5)          │            30 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 219,542 (857.59 KB)

 Trainable params: 219,542 (857.59 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Block 3: Define Custom Masked Loss Function
# Purpose: Create a loss function that only calculates error on the masked timesteps.
# This focuses the model's training on the "fill in the blanks" task.

print("--- Block 3: Defining Custom Masked Loss Function ---")

def masked_mse_loss(y_true, y_pred):
    """
    Calculates Mean Squared Error only on the parts of the sequence that were masked.
    y_true is the original sequence.
    The input to the model (which we don't see here) was the masked sequence.
    We can identify the masked parts by finding where the model's input would have been zero.
    """
    # Create a mask by checking where the true values are non-zero
    # This assumes that a feature vector of all zeros is a mask.
    # We create a mask for each feature, then check if ANY feature was non-zero.
    mask = tf.cast(tf.reduce_any(y_true != 0, axis=-1), dtype=tf.float32)

    # Invert the mask to find where the inputs WERE masked (i.e., where they were zero)
    # This is a bit of a trick: we want to penalize error where the input was zero,
    # but the `y_true` we get here is the *original*, non-zero data.
    # A better way is to compare the prediction to the original and only calculate
    # loss on the timesteps that were masked. We need to know which ones they were.
    # The generator provides both, but the model's `fit` method does not pass it here.

    # A simpler and more robust approach for this autoencoder:
    # Just compute the reconstruction loss over the whole sequence.
    # The model will still be forced to learn the structure to reconstruct the masked parts.
    # Let's stick to standard MSE for simplicity and robustness in this first version.
    # A custom loss would be needed if we passed the mask itself to the loss function.

    return tf.keras.losses.mean_squared_error(y_true, y_pred)

# Re-compile the model with the chosen loss function
autoencoder.compile(optimizer='adam', loss='mse') # Using standard MSE is a strong baseline.
print("Model compiled with 'mean_squared_error' loss function.")

# --- End of Block 3 ---
print("\n" + "="*50 + "\n")

--- Block 3: Defining Custom Masked Loss Function ---
Model compiled with 'mean_squared_error' loss function.




In [ ]:
# Block 4: Train the Model
# Purpose: Train the autoencoder on the preprocessed data.

print("--- Block 4: Training the Model ---")

# Define callbacks to save the best model and stop early if validation loss doesn't improve
checkpoint = ModelCheckpoint('best_autoencoder.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min',
                             verbose=1)

early_stopping = EarlyStopping(monitor='val_loss',
                              patience=5,
                              mode='min',
                              restore_best_weights=True,
                              verbose=1)

# Train the model
history = autoencoder.fit(train_dataset,
                          epochs=50, # Set a high number, early stopping will find the best
                          validation_data=val_dataset,
                          callbacks=[checkpoint, early_stopping])

print("\nTraining complete.")

# --- End of Block 4 ---
print("\n" + "="*50 + "\n")

--- Block 4: Training the Model ---
Epoch 1/50
   1077/Unknown 146s 127ms/step - loss: 0.6944

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 1: val_loss improved from inf to 0.42519, saving model to best_autoencoder.h5


1077/1077 ━━━━━━━━━━━━━━━━━━━━ 156s 136ms/step - loss: 0.6943 - val_loss: 0.4252
Epoch 2/50
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.3982
Epoch 2: val_loss improved from 0.42519 to 0.35913, saving model to best_autoencoder.h5


1077/1077 ━━━━━━━━━━━━━━━━━━━━ 146s 134ms/step - loss: 0.3981 - val_loss: 0.3591
Epoch 3/50
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.3459
Epoch 3: val_loss improved from 0.35913 to 0.31689, saving model to best_autoencoder.h5


...

1077/1077 ━━━━━━━━━━━━━━━━━━━━ 144s 132ms/step - loss: 0.0672 - val_loss: 0.0669
Epoch 26/50
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.0660
Epoch 26: val_loss improved from 0.06692 to 0.06580, saving model to best_autoencoder.h5


1077/1077 ━━━━━━━━━━━━━━━━━━━━ 144s 132ms/step - loss: 0.0660 - val_loss: 0.0658
Epoch 27/50
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.0651
Epoch 27: val_loss did not improve from 0.06580
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 144s 132ms/step - loss: 0.0651 - val_loss: 0.0684
Epoch 28/50
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - loss: 0.0639
Epoch 28: val_loss improved from 0.06580 to 0.06542, saving model to best_autoencoder.h5


1077/1077 ━━━━━━━━━━━━━━━━━━━━ 144s 132ms/step - loss: 0.0639 - val_loss: 0.0654
Epoch 29/50
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.0633
Epoch 29: val_loss improved from 0.06542 to 0.06413, saving model to best_autoencoder.h5


1077/1077 ━━━━━━━━━━━━━━━━━━━━ 145s 133ms/step - loss: 0.0633 - val_loss: 0.0641
Epoch 30/50
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.0625
Epoch 30: val_loss improved from 0.06413 to 0.06364, saving model to best_autoencoder.h5


1077/1077 ━━━━━━━━━━━━━━━━━━━━ 146s 134ms/step - loss: 0.0625 - val_loss: 0.0636
Epoch 31/50
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.0625
Epoch 31: val_loss did not improve from 0.06364
1077/1077 ━━━━━━━━━━━━━━━━━━━━ 145s 133ms/step - loss: 0.0625 - val_loss: 0.0643
Epoch 32/50
 161/1077 ━━━━━━━━━━━━━━━━━━━━ 1:54 125ms/step - loss: 0.0597

In [ ]:
# Block 5: Plot Training History and Save the Encoder
# Purpose: Visualize the training process and save the final, pre-trained encoder model.

print("--- Block 5: Saving the Pre-trained Encoder ---")

# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training History')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error Loss')
plt.legend()
plt.grid(True)
plt.savefig('training_history.png')
print("Saved training history plot to 'training_history.png'")
plt.show()

# The most important step: save the pre-trained encoder part of the model
encoder.save('gaze_encoder_pretrained.h5')
print("\nPre-trained encoder saved to 'gaze_encoder_pretrained.h5'")
print("This encoder is now ready for Phase 2: Fine-tuning!")

# --- End of Block 5 ---

## 6. Evaluation and Saving

In [ ]:
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import mse, MeanSquaredError

print("--- Block 1: Loading Models and Data ---")

try:
    # Define custom objects dictionary to help Keras recognize the 'mse' function
    custom_objects = {
        'mse': mse,
        'MeanSquaredError': MeanSquaredError
    }

    # Load the models with custom objects
    autoencoder = load_model('/content/best_autoencoder.h5', custom_objects=custom_objects)
    encoder = load_model('/content/gaze_encoder_pretrained.h5', custom_objects=custom_objects)

    # Load the scaler
    with open('/content/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    # Load the test data
    X_test = np.load('/content/X_test.npy')

    print("Successfully loaded all necessary assets.")
    print(f"Encoder input shape: {encoder.input_shape}")
    print(f"Encoder output shape: {encoder.output_shape}")
    print(f"Test data shape: {X_test.shape}")

except Exception as e:
    print(f"An error occurred loading the files. Make sure all files are present. Details: {e}")

# --- End of Block 1 ---
print("\n" + "="*50 + "\n")

In [ ]:
# Block 2: Qualitative Test - Visualizing Reconstruction
# Purpose: To see how well the model can reconstruct a masked sequence.

print("--- Block 2: Visualizing Reconstruction on a Test Sample ---")

# Take one sample sequence from the test set
sample_sequence = X_test[100] # Using the 101st sample

# Create a masked version of the sample
masked_sample = sample_sequence.copy()
seq_len, n_features = sample_sequence.shape
n_to_mask = int(np.ceil(seq_len * 0.20)) # Mask 20%
mask_indices = np.random.choice(seq_len, n_to_mask, replace=False)
masked_sample[mask_indices] = 0.0

# The model expects a batch, so we add a batch dimension
masked_sample_batch = np.expand_dims(masked_sample, axis=0)

# Get the model's reconstruction
reconstructed_sample_batch = autoencoder.predict(masked_sample_batch)
reconstructed_sample = reconstructed_sample_batch[0]

# --- Plotting the results ---
feature_names = ['Duration', 'X-Pos', 'Y-Pos', 'Sac Amplitude', 'Sac Velocity']
fig, axes = plt.subplots(n_features, 1, figsize=(15, 12), sharex=True)
fig.suptitle('Qualitative Reconstruction Test', fontsize=16)

for i in range(n_features):
    axes[i].plot(sample_sequence[:, i], label='Original', color='blue', marker='.', linestyle='--')
    axes[i].plot(reconstructed_sample[:, i], label='Reconstructed', color='red', marker='x')
    axes[i].set_ylabel(feature_names[i])
    # Highlight the masked areas
    for idx in mask_indices:
        axes[i].axvspan(idx - 0.5, idx + 0.5, color='gray', alpha=0.3, label='Masked' if idx==mask_indices[0] else "")

axes[-1].set_xlabel('Timestep (Fixation Number)')
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('reconstruction_test.png')
print("Saved plot: reconstruction_test.png")
plt.show()

# --- End of Block 2 ---
print("\n" + "="*50 + "\n")